## Google Colab configs

In [3]:
!git clone https://github.com/ReverendBayes95/Amazon_Reviews.git
%cd Amazon_Reviews

Cloning into 'Amazon_Reviews'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 50 (delta 19), reused 35 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 72.91 KiB | 18.23 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/Amazon_Reviews


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = "/content/drive/MyDrive/amazon_reviews_dataset/"

## Import libraries

In [11]:
import utils.data_io as data_io
import utils.filters as filters
import utils.aggregations as aggregations
import utils.models as models
import variables
import torch
import pandas as pd

## Read the reviews data

In [5]:
colnames = ['polarity', 'title', 'text']
train_df = data_io.read_reviews(path + "train.csv", colnames)
test_df = data_io.read_reviews(path + "test.csv", colnames)

In [6]:
train = aggregations.stratified_sample(train_df, 'polarity', 100000)
test = aggregations.stratified_sample(test_df, 'polarity', 10000)

del train_df, test_df

## Pre-process the reviews

In [7]:
train_pre_process = filters.TextPreProcess(train, variables.appos)
test_pre_process = filters.TextPreProcess(test, variables.appos)
train = train_pre_process.apply_text_filtering()
test = test_pre_process.apply_text_filtering()

train = aggregations.create_single_review(train)
test = aggregations.create_single_review(test)

## Predict the sentiment

### With transformers pipelines module

In [8]:
pipeline_preds = models.hf_classification_pipeline(test)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [19]:
pipeline_preds = pd.DataFrame(pipeline_preds)\
                  .rename(columns={'label': 'pl_label', 'score': 'pl_score'})\

pipeline_preds['pl_label'] = pipeline_preds['pl_label'].replace({"NEGATIVE": 1, "POSITIVE": 2})
pipeline_preds.index = test.index
test = test.join(pipeline_preds)

/tmp/ipython-input-19-470836371.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pipeline_preds['pl_label'] = pipeline_preds['pl_label'].replace({"NEGATIVE": 1, "POSITIVE": 2})


### Calculate accuracy achieved with the raw pipelines module

In [33]:
sum(test['polarity'].eq(test['pl_label']))*100/(len(test))

89.47894789478948